In [20]:
import common
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import os, pathlib, enum
import json
from pathlib import Path
from sqlalchemy import text
from sqlalchemy.exc import IntegrityError
import psycopg2
import matplotlib.pyplot as plt
from IPython.display import Markdown
import numpy as np
import math

database = common.Database()
pcaps = pd.read_sql("SELECT * FROM pcap", database.engine)
malwares = pd.read_sql("""
SELECT
	MW.*,
	SUM(PCAP.U) AS U
FROM
	MALWARE AS MW
	JOIN PCAP ON MW.ID = PCAP.MALWARE_ID GROUP BY MW.ID
order by u
""", database.engine).set_index("id")

In [19]:
bdn = pd.read_sql("""
SELECT
    PCAP.MALWARE_ID,
    SUM(PCAP.U) AS U,
    DN.DN,
    WL.RANK_BDN,
    COUNT(*) AS APP,
    SUM(CASE WHEN M.RCODE = 3 THEN 1 ELSE 0 END) AS NX,
    COUNT(DISTINCT M.PCAP_ID) AS PCAPS,
    DN_NN1.value as epsilon
FROM
    (SELECT * FROM MESSAGE WHERE IS_R is TRUE) AS M
    JOIN PCAP ON M.PCAP_ID = PCAP.ID
    JOIN DN ON M.DN_ID = DN.ID
    JOIN WHITELIST_DN WL ON WL.DN_ID = DN.ID
    JOIN (SELECT * FROM DN_NN WHERE NN_ID=1) AS DN_NN1 ON DN.id=DN_NN1.DN_ID
    WHERE
    PCAP.INFECTED is true AND PCAP.DATASET='CTU-13'
GROUP BY
    PCAP.MALWARE_ID,
    DN.DN,
    WL.RANK_BDN,
    DN_NN1.VALUE;
""", database.engine.connect())

In [30]:
mwlist = pcaps[(pcaps["dataset"] == "CTU-13")][["id", "malware_id"]].groupby("malware_id").agg({"id": lambda x: x.to_list()}).sort_values(by="malware_id")
mwdga = {}


def genmask(mwid, appgt10=None, nx_gt0=None, rankbdn_null=None, nx_app_gt=None, nx_app_lt=None, th=None):
    df = bdn.copy()
    df["nx/app"] = df["nx"] / df["app"]
    tmp = []
    mask = pd.Series(np.ones(df.shape[0], dtype=bool))
    if appgt10 is not None:
        tmp.append("app > 10" if appgt10 else "app <= 10")
        mask &= df.app > 10 if appgt10 else df.app <= 10
    if nx_gt0 is not None:
        tmp.append("nx > 0" if nx_gt0 else "nx == 0")
        mask &= df.nx > 0 if nx_gt0 else df.nx == 0
    if rankbdn_null is not None:
        tmp.append("rankbdn null" if rankbdn_null else " rankbdn not null")
        mask &= df.rank_bdn.isna() if rankbdn_null else (~df.rank_bdn.isna())
    if nx_app_gt is not None:
        tmp.append(f"nx/app > {nx_app_gt}")
        mask &= df["nx/app"] > nx_app_gt
    if nx_app_lt is not None:
        tmp.append(f"nx/app < {nx_app_lt}")
        mask &= df["nx/app"] < nx_app_lt
    if th:
        tmp.append(f"epsilon > {th}")
        mask &= df["epsilon"] > th
    df = df[(df.malware_id == mwid) & mask]

    tmp = [f"count: {df.shape[0]}", f"uniques: {int(malwares.loc[mwid,"u"])}"] + tmp# type: ignore
    display(Markdown("- " + "\n- ".join(tmp)))
    if df.shape[0]:
        display(df.sort_values(by="app", ascending=False))
    else:
        display(Markdown("> No data"))
    display(Markdown("***"))
    return df

mwlist

,id
malware_id,
1,"[15, 18, 23, 11, 9, 19, 21, 17, 22, 10, 14, 20..."
177,[57]
178,[37]
179,[41]
180,[54]
181,"[44, 43]"
182,[34]
183,[27]
184,[36]


In [31]:
mwid = 177
display(Markdown(f"# {mwid}"))

genmask(mwid, appgt10=False, nx_app_lt=0.2)
genmask(mwid, appgt10=False, nx_app_gt=0.8)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "There are domains very DGA-like"
    ],
    "dga": 2
}

display(Markdown(f"\n# DGA {mwdga[mwid]["dga"]} for {mwid}"))

# 177

- count: 14
- uniques: 2027
-  rankbdn not null

,malware_id,u,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
257,177,58783.0,dns.msftncsi.com,453160.0,29,3,1,2.874122e-07,0.103448
1765,177,34459.0,www.bing.com,156.0,17,1,1,3.527413e-07,0.058824
1766,177,28378.0,www.google.com,3.0,14,8,1,3.055666e-07,0.571429
1767,177,28378.0,www.google.cz,3963.0,14,8,1,2.709334e-04,0.571429
29,177,6081.0,aol.com,1820.0,3,0,1,6.206943e-05,0.000000
578,177,6081.0,hotmail.com,3910.0,3,0,1,5.878788e-08,0.000000
1008,177,6081.0,mx1.hotmail.com,3910.0,3,0,1,4.716464e-08,0.000000
1009,177,6081.0,mx2.hotmail.com,3910.0,3,0,1,5.496674e-08,0.000000
1715,177,6081.0,web.de,8448.0,3,0,1,4.791703e-06,0.000000
1880,177,6081.0,yahoo.com,228.0,3,0,1,4.719932e-04,0.000000


***

- count: 46
- uniques: 2027
- app <= 10
- nx/app < 0.2

...

***

- count: 1788
- uniques: 2027
- app <= 10
- nx/app > 0.8

,malware_id,u,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
0,177,14189.0,adedtppnnfmztussktlnbknkn.org,NaN,7,7,1,1.000000,1.000000
886,177,14189.0,lrtofahqzlvrsxsscdaykzuqs.info,NaN,7,6,1,1.000000,0.857143
971,177,14189.0,mjrkqsdikbdibqhqprwoplqkvkzyt.info,NaN,7,7,1,1.000000,1.000000
964,177,14189.0,mhealwgzlnfljmbmfiramtdttw.biz,NaN,7,7,1,1.000000,1.000000
963,177,14189.0,mfvcijdeyguhadxknzfqx.org,NaN,7,7,1,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...
501,177,2027.0,haknbknizbbinqsgyuwlvdqoay.info,NaN,1,1,1,1.000000,1.000000
1493,177,2027.0,tghimrwcaelvxgllylwklrsbulgi.biz,NaN,1,1,1,1.000000,1.000000
1604,177,2027.0,uogyeqobdirxoxsaikfdscirmnbufy.com,NaN,1,1,1,0.999999,1.000000
515,177,2027.0,hbqvkdqkydxxxobyhetzdbamkr.info,NaN,1,1,1,1.000000,1.000000


***


# DGA 2 for 177

In [ ]:
mwid = 178
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, appgt10=None, nx_app_gt=0.8)
genmask(mwid, appgt10=None, nx_app_lt=0.2)
genmask(mwid, appgt10=None, th=0.99)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "There are domains very DGA-like.",
    ],
    "dga": 2
}

display(Markdown(f"\n# DGA {mwdga[mwid]["dga"]} for {mwid}"))

In [ ]:
# mwid = mwlist.index[2]
mwid = 179
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=False, appgt10=True)
genmask(mwid, rankbdn_null=False, nx_gt0=True)
genmask(mwid, rankbdn_null=True, appgt10=None, nx_app_gt=0.8)
genmask(mwid, rankbdn_null=True, appgt10=None, nx_app_lt=0.2)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn.",
        "There are many domains very DGA-like.",
    ],
    "dga": 2
}

In [ ]:
# mwid = mwlist.index[3]
mwid = 180
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=False, appgt10=None)
genmask(mwid, rankbdn_null=False, nx_gt0=True)
genmask(mwid, rankbdn_null=True, appgt10=None, nx_app_gt=0.8)
genmask(mwid, rankbdn_null=True, appgt10=None, nx_app_lt=0.2)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "There are many domains very DGA-like.",
    ],
    "dga": 2
}

In [ ]:
# mwid = mwlist.index[4]
mwid = 181
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=False, nx_gt0=True)
genmask(mwid, rankbdn_null=True, appgt10=False, nx_app_gt=0.8)
genmask(mwid, rankbdn_null=True, appgt10=False, nx_app_lt=0.2)
genmask(mwid, rankbdn_null=True, appgt10=True, nx_app_gt=0.8)
genmask(mwid, rankbdn_null=False, appgt10=True, nx_app_lt=0.5)
genmask(mwid, rankbdn_null=True, appgt10=False, nx_app_lt=0.5)
genmask(mwid, rankbdn_null=None, appgt10=False, nx_app_lt=None)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s, many of them have thousands of appereances with an nx/app > 0.1.",
        "No domains very DGA-like.",
        "Seems to use composed domains, like dynamic-dns and a not very _fanciful_ subdomain.",
        "There are no domains apperead less than 10 times."
    ],
    "dga": 1
}

In [ ]:
# mwid = mwlist.index[5]
mwid = 182
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=None, nx_gt0=True)
genmask(mwid, rankbdn_null=True, appgt10=False, nx_app_gt=0.8)
genmask(mwid, rankbdn_null=True, nx_gt0=True, th=0.999)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Very few NX.",
    ],
    "dga": 1
}

In [ ]:
# mwid = mwlist.index[6]
mwid = 183
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=False, appgt10=None)
genmask(mwid, rankbdn_null=None, nx_gt0=True)
genmask(mwid, rankbdn_null=True, appgt10=False, nx_app_gt=0.8)
genmask(mwid, rankbdn_null=True, nx_gt0=True, th=0.999)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "There are 4 whitelisted bdn.",
        "Many DGA dn using human-readable words.",
    ],
    "dga": 2,
    "dga-subtype": "human-readable"
}

In [ ]:
# mwid = mwlist.index[7]
mwid = 184
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=None, nx_gt0=True)
genmask(mwid, rankbdn_null=True, appgt10=False, nx_app_gt=0.8)
genmask(mwid, rankbdn_null=True, appgt10=None, nx_app_lt=0.4)
genmask(mwid, rankbdn_null=True, nx_gt0=True, th=0.999)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Not using DGA dns.",
    ],
    "dga": 1
}

In [ ]:
mwid = mwlist.index[8]
mwid = 185
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=False, appgt10=None)
genmask(mwid, rankbdn_null=True, nx_gt0=True)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Just one NX response for `arlingtontimberframes.com`, which appears 1 time.",
    ],
    "dga": 1
}

In [ ]:
# mwid = mwlist.index[9]
mwid = 186
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many spam dns, but not DGA.",
    ],
    "dga": 1
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523
# mwid = mwlist.index[10]
mwid = 187
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many spam dns, but not DGA.",
        "Use of bdn `alacartebelini.com` with many subdomains which could be DGA but made of numbers.",
        f"{alac.shape[0]} bdn using mostly `alacartebelini.com` and having nx/app = 0:",
        alac.dn.apply(lambda x: ".".join(x.split(".")[-2:])).value_counts().to_markdown()
    ],
    "dga": 1
}

alac.dn.apply(lambda x: ".".join(x.split(".")[-2:])).value_counts()

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523
# mwid = mwlist.index[11]
mwid = 189
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many spam dns, but not DGA.",
    ],
    "dga": 1
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523
# mwid = mwlist.index[12]
mwid = 190
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of DGA-like domains with nx/app > 0.9.",
    ],
    "dga": 2
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523 ]
# mwid = mwlist.index[12]
mwid = 191
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
df = genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
genmask(mwid, rankbdn_null=True, nx_gt0=True, th=0.999)


mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many spam domains.",
    ],
    "dga": 1
}


In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523 ]
# mwid = mwlist.index[13]
mwid = 193
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many DGA-like domains.",
    ],
    "dga": 2
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523 ]
# mwid = mwlist.index[14]
mwid = 194
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many spam domains.",
    ],
    "dga": 1
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523 ]
# mwid = mwlist.index[15]
mwid = 195
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many DGA-like domains.",
    ],
    "dga": 2
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523 ]
# mwid = mwlist.index[16]
mwid = 196
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=None)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many spam domains.",
    ],
    "dga": 1
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523 ]
# mwid = mwlist.index[17]
mwid = 197
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=None)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many spam domains.",
    ],
    "dga": 1
}

In [ ]:
# [177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 523 ]
# mwid = mwlist.index[18]
mwid = 523
display(Markdown(f"# {mwid}"))

wt = genmask(mwid, rankbdn_null=False)
genmask(mwid, rankbdn_null=True, nx_gt0=True)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=False)
alac = genmask(mwid, rankbdn_null=True, nx_gt0=None)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)
genmask(mwid, rankbdn_null=True, nx_app_gt=0.9)

mwdga[mwid] = {
    "notes": [
        f"There are {wt.shape[0]} whitelisted bdn-s.",
        "Use of many DGA domains.",
    ],
    "dga": 2
}

In [ ]:
display(Markdown("# Overall"))

for mwid in mwdga:
    display(Markdown(f"## {mwid}: dga={mwdga[mwid]["dga"]} {'(%s)' % mwdga[mwid]['dga-subtype'] if 'dga-subtype' in mwdga[mwid] else ''}"))
    for note in mwdga[mwid]["notes"]:
        display(Markdown(note))


In [ ]:

mwdga_iddga = []
for mwid in mwdga:
    mwdga_iddga.append([mwid, mwdga[mwid]["dga"], mwdga[mwid]['dga-subtype'] if 'dga-subtype' in mwdga[mwid] else None])
    pass

dfmw = pd.DataFrame(mwdga_iddga, columns=["id", "dga", "dga-subtype"])

for _, row in dfmw.iterrows():
    with database.engine.connect() as conn:
        try:
            conn.execute(
                text("""
                UPDATE MALWARE SET DGA=:dga WHERE id=:id;
                """),
                [{"id": row["id"], "dga": row["dga"] }]
            )
            conn.commit()
        except IntegrityError as e:
            try:
                raise e.orig
            except psycopg2.errors.UniqueViolation as e:
                pass
            except:
                print(e)



In [ ]:
display(Markdown("# Test with db"))

pcaps = pd.read_sql("SELECT * FROM pcap", database.engine)
malwares = pd.read_sql("SELECT * FROM malware", database.engine).set_index("id")

pcaps = pcaps[(pcaps.dataset == "CTU-13") & (pcaps.infected)]

alls = []
for _, row in pcaps.iterrows():
    mwid = row["malware_id"]
    alls.append(malwares.loc[mwid, "dga"] == mwdga[mwid]["dga"])

display(Markdown("Success" if all(alls) else "Failed"))


In [6]:
display(Markdown("# 1"))

genmask(1, appgt10=None, th=0.5)

# 1

- epsilon > 0.5
- count: 0

> No data

***

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app


In [8]:
display(Markdown("# 177"))
_ = genmask(177, appgt10=None, th=0.99)


# 177

- epsilon > 0.99
- count: 2001

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
1939,177,ytnjqgmnhrxeathuwkvgvcts.org,NaN,8,4,1,1.000000,0.5
0,177,adedtppnnfmztussktlnbknkn.org,NaN,7,7,1,1.000000,1.0
888,177,lryhvkcsotlvhktjffuaioft.com,NaN,7,7,1,0.999999,1.0
895,177,lthnvbepjvworeawsvoozeagq.com,NaN,7,7,1,1.000000,1.0
903,177,lvdojscdynzmrxzxtzzvkpneu.biz,NaN,7,7,1,0.999999,1.0
...,...,...,...,...,...,...,...,...
1959,177,zdhzsktzhustdxbidyemtga.biz,NaN,1,0,1,1.000000,0.0
1460,177,svbaormntxcmofzwpfkrpuwtg.com,NaN,1,1,1,1.000000,1.0
485,177,gyaiyhtodaljdmugpfnfmfgabirsztdy.biz,NaN,1,1,1,1.000000,1.0
90,177,biolzcmemlvtgizxvbuumnba.com,NaN,1,1,1,0.999999,1.0


***

In [9]:
display(Markdown("# 178"))
_ = genmask(178, appgt10=None, th=0.99)

# 178

- epsilon > 0.99
- count: 973

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
2034,178,bbtybrmepmru.xyz,NaN,1046,1046,1,0.999963,1.0
2791,178,rkpiqcmmmbue.xyz,NaN,1031,1031,1,0.999993,1.0
2563,178,jxihgmogkkgs.xyz,NaN,1029,1029,1,1.000000,1.0
2091,178,brbqlwsepmtm.xyz,NaN,1029,1029,1,0.999998,1.0
2047,178,bcxuxooveffi.xyz,NaN,1029,1029,1,0.999632,1.0
...,...,...,...,...,...,...,...,...
2337,178,ghpeyebtuurv.xyz,NaN,340,340,1,0.999990,1.0
2874,178,stqbynwckcsx.xyz,NaN,339,339,1,1.000000,1.0
2794,178,rlkkethqqvbt.xyz,NaN,338,338,1,1.000000,1.0
2906,178,uipojtchwbur.xyz,NaN,338,338,1,0.999945,1.0


***

In [10]:
display(Markdown("# 179"))
_ = genmask(179, appgt10=None, th=0.99)

# 179

- epsilon > 0.99
- count: 2520

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
3192,179,bllewbrb.bit,NaN,690,0,1,0.999849,0.000000
5597,179,ygwnfft.sh,NaN,569,0,1,0.999980,0.000000
4911,179,rlbdexogfcup.com,NaN,16,0,1,0.999994,0.000000
4669,179,oxqmjsmrdrlartl.eu,NaN,6,0,1,1.000000,0.000000
4527,179,npkxghmoru.biz,NaN,3,1,1,0.995845,0.333333
...,...,...,...,...,...,...,...,...
3936,179,icdvvugfns.eu,NaN,1,1,1,0.995881,1.000000
3937,179,icfvnisevkrskwdj.pw,NaN,1,1,1,0.999995,1.000000
3938,179,icgdhalsxvrfhdiae.org,NaN,1,1,1,0.999997,1.000000
3939,179,ichlmgvxpsw.ki,NaN,1,1,1,1.000000,1.000000


***

In [14]:
display(Markdown("# 180"))
df = genmask(180, appgt10=None, th=0.99)

# 180

- epsilon > 0.99
- count: 3603

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
9701,180,pxbv2w0wlhxa.smis.cc,NaN,331,0,1,0.999999,0.000000
7318,180,a3ynj0a9uq85ct4.pcg.su,NaN,329,0,1,0.999850,0.000000
10115,180,sgaw8lg.smis.cc,NaN,291,1,1,0.999827,0.003436
6378,180,4c5a230b5iwb8vxvq.smis.cc,NaN,265,0,1,0.999999,0.000000
7992,180,eud5qch6ss8ix.smis.cc,NaN,256,1,1,1.000000,0.003906
...,...,...,...,...,...,...,...,...
7754,180,df4izrrpzw04.smis.cc,NaN,1,1,1,0.999983,1.000000
7757,180,df7zd2e0.smis.cc,NaN,1,1,1,0.997656,1.000000
7758,180,df9urwzbhdnvxn.ccl.su,NaN,1,1,1,1.000000,1.000000
7759,180,df9uuno6v5bw.smis.cc,NaN,1,1,1,1.000000,1.000000


***

In [15]:
display(Markdown("# 181"))
df = genmask(181, appgt10=None, th=0.99)

# 181

- epsilon > 0.99
- count: 3

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
11429,181,coopvgg.com.ar,5495614.0,11587,3769,2,0.999727,0.325278
11882,181,wvwxuip.ukk,NaN,6228,1878,2,0.999994,0.301541
11417,181,cisb.com.ar,NaN,6086,2033,2,0.992587,0.334045


***

In [16]:
display(Markdown("# 182"))
df = genmask(182, appgt10=None, th=0.99)

# 182

- epsilon > 0.99
- count: 36

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
14971,182,mail.utanet.at,2055189.0,509,1,1,0.996140,0.001965
13590,182,imap.station.cz,NaN,21,0,1,0.994687,0.000000
12034,182,360grad.360kommunikation.biz,NaN,11,0,1,0.999981,0.000000
17442,182,taarnby-cykler.dk,NaN,10,0,1,0.999720,0.000000
13613,182,imapproxy2.dd24.net,NaN,5,0,1,0.999957,0.000000
13603,182,imap4.amis.net,517602.0,5,0,1,0.992898,0.000000
13609,182,imap7.amis.net,517602.0,5,0,1,0.999435,0.000000
18229,182,xmail.xpirio.net,NaN,4,0,1,0.994011,0.000000
12735,182,cvorago.dnssw.net,NaN,4,2,1,0.999993,0.500000
13606,182,imap5.amis.net,517602.0,4,0,1,0.996852,0.000000


***

In [17]:
display(Markdown("# 183"))
df = genmask(183, appgt10=None, th=0.99)

# 183

- epsilon > 0.99
- count: 2

,malware_id,dn,rank_bdn,app,nx,pcaps,epsilon,nx/app
18359,183,schedulerecommend.com,NaN,228,118,1,0.999731,0.517544
18303,183,confidenceconcentrate.com,NaN,201,180,1,0.998932,0.895522


***

In [ ]:
display(Markdown(184))
genmask(184, appgt10=None, th=0.99)

In [ ]:
display(Markdown(185))
genmask(185, appgt10=None, th=0.99)

In [ ]:
display(Markdown(186))
genmask(186, appgt10=None, th=0.99)

In [ ]:
display(Markdown(187))
genmask(187, appgt10=None, th=0.99)

In [ ]:
display(Markdown(189))
genmask(189, appgt10=None, th=0.99)

In [ ]:
display(Markdown(190))
genmask(190, appgt10=None, th=0.99)

In [ ]:
display(Markdown(191))
genmask(191, appgt10=None, th=0.99)

In [ ]:
display(Markdown(193))
genmask(193, appgt10=None, th=0.99)

In [ ]:
display(Markdown(194))
genmask(194, appgt10=None, th=0.99)

In [ ]:
display(Markdown(195))
genmask(195, appgt10=None, th=0.99)

In [ ]:
display(Markdown(196))
genmask(196, appgt10=None, th=0.99)

In [ ]:
display(Markdown(197))
genmask(197, appgt10=None, th=0.99)

In [ ]:
display(Markdown(523))
genmask(523, appgt10=None, th=0.99)

In [32]:


for mwid in mwlist.index:
    print(f"""{{
    "cell_type": "code",
    "execution_count": null,
    "id": "c27d79d6",
    "metadata": {{}},
    "outputs": [],
    "source": [
            "mwid = {mwid}\\n",
            "display(Markdown(\\\"# {{mwid}}\\\"))\\n",
            "\\n",
            "genmask({{mwid}}, appgt10=None, th=0.5)\\n",
            "\\n",
            "mwdga[mwid] = 1/0\\n",
            "\\n"
    ]
    }},""")

{
    "cell_type": "code",
    "execution_count": null,
    "id": "c27d79d6",
    "metadata": {},
    "outputs": [],
    "source": [
            "mwid = 1\n",
            "display(Markdown("# 1"))\n",
            "\n",
            "genmask(1, appgt10=None, th=0.5)\n",
            "\n",
            "mwdga[mwid] = 1/0\n",
            "\n"
    ]
    },
{
    "cell_type": "code",
    "execution_count": null,
    "id": "c27d79d6",
    "metadata": {},
    "outputs": [],
    "source": [
            "mwid = 177\n",
            "display(Markdown("# 177"))\n",
            "\n",
            "genmask(177, appgt10=None, th=0.5)\n",
            "\n",
            "mwdga[mwid] = 1/0\n",
            "\n"
    ]
    },
{
    "cell_type": "code",
    "execution_count": null,
    "id": "c27d79d6",
    "metadata": {},
    "outputs": [],
    "source": [
            "mwid = 178\n",
            "display(Markdown("# 178"))\n",
            "\n",
            "genmask(178, appgt10=None, th=0.5)\n",
            "\n